In [1]:
# Importamos el handle (MLClient)
from azure.ai.ml import MLClient

# Importamos también del paquete de azure.identity
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

#vamos a autenticar directamente con el navegador

credential = InteractiveBrowserCredential()

In [2]:
# Una vez autenticados, conectamos al Workspace de Azure Machine Learning. Podemos usar el config.json 
# o incluirlo aquí
ml_client = MLClient(
    credential=credential,
    subscription_id="3382d9b3-bbe5-4548-ad6a-029648476060",
    resource_group_name="TFM_AI",
    workspace_name="aml_tfm",
)
print (ml_client)

MLClient(credential=<azure.identity._credentials.browser.InteractiveBrowserCredential object at 0x107dddd50>,
         subscription_id=3382d9b3-bbe5-4548-ad6a-029648476060,
         resource_group_name=TFM_AI,
         workspace_name=aml_tfm)


In [3]:
# Importamos las librerías necesarias para trabajar con dataset
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

web_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"

credit_data = Data(
    name="tarjetas_credito_default",
    path=web_path,
    type=AssetTypes.URI_FILE,
    description="Dataset para predecir impago en tarjetas de crédito",
    tags={"source_type": "web", "source": "UCI ML Repo"},
    version="1.0.0",
)

In [22]:
credit_data = ml_client.data.create_or_update(credit_data)
print(
    f"Dataset with name {credit_data.name} was registered to workspace, the dataset version is {credit_data.version}"
)

Dataset with name tarjetas_credito_default was registered to workspace, the dataset version is 1.0.0


In [25]:
#puedo acceder a este dataset asignandolo a una variable de la siguient forma
credit_dataset = ml_client.data.get("tarjetas_credito_default", version='1.0.0')

In [30]:
from azure.ai.ml.entities import AmlCompute

cpu_compute_target = "tfm-cluster"

try:
    # miramos si existe ya un cluster con el mismo nombre
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"Ya existe un cluster con nombre {cpu_compute_target},lo utilizaremos como está creado."
    )

except Exception:
    print("Creando un nuevo compute cluster...")

    # Creamos el Azure ML Compute Cluster con los parámetros indicados
    cpu_cluster = AmlCompute(
        # Nombre del cluster
        name="tfm-cluster",
        # Azure ML Compute es el servicio On-demand de cómputo
        type="amlcompute",
        # VM Serie que vamos a utilizar. Si fuese necesario un clúster con GPU deberíamos utilizar otra serie
        size="STANDARD_DS2_V2",
        # Número de nodos mínimo en ejecución cuando no hay jobs a ejecutar
        min_instances=0,
        # Número máximo de nodos del auto escalado.
        max_instances=4,
        # Tiempo que se espera desde que termina el job hasta que se vuelve a desescalar.
        idle_time_before_scale_down=180,
        # Con este parámetro podemos indicar si queremos utilizar VM de tipo Low Priority o Regular PAYG
        tier="Dedicated",
    )

    # Ahora le pasamos al handler de ML Client el objeto para que lo actualice y cree.
    cpu_cluster = ml_client.begin_create_or_update(cpu_cluster)

print(
    f"AMLCompute con nombre {cpu_cluster.name} se ha creado. Instancia tipo: {cpu_cluster.size}"
)

Creando un nuevo compute cluster...
AMLCompute con nombre tfm-cluster se ha creado. Instancia tipo: STANDARD_DS2_V2


In [31]:
import os

dir_dependencias = "./dependencias"
os.makedirs(dir_dependencias, exist_ok=True)

In [32]:
%%writefile {dir_dependencias}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0

Writing ./dependencias/conda.yml


In [33]:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-python-tfm"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Entorno creado para el TFM para trabajar con el dataset Credit Card Defaults pipeline",
    tags={"scikit-learn": "0.24.2"},
    conda_file=os.path.join(dir_dependencias, "conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
    version="1.0",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment con nombre {pipeline_job_env.name} se ha registrado. Version: {pipeline_job_env.version}"
)

Environment con nombre aml-python-tfm se ha registrado. Version: 1.0


In [35]:
import os

data_prep_src_dir = "./componentes/data_prep"
os.makedirs(data_prep_src_dir, exist_ok=True)

In [36]:
%%writefile {data_prep_src_dir}/data_prep.py
import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
import logging
import mlflow


def main():
    """Funcion principal del script"""

    # Argumentos de entrada/Salida
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="Path a los datos de entrada")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--train_data", type=str, help="path a los datos de entrenamiento")
    parser.add_argument("--test_data", type=str, help="path a los datos de test")
    args = parser.parse_args()

    # Empezamos el logging 
    mlflow.start_run()

    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    print("Datos de entrada", args.data)
    # Creamos nuestro dataframe
    credit_df = pd.read_excel(args.data, header=1, index_col=0)

    mlflow.log_metric("num_samples", credit_df.shape[0])
    mlflow.log_metric("num_features", credit_df.shape[1] - 1)

    credit_train_df, credit_test_df = train_test_split(
        credit_df,
        test_size=args.test_train_ratio,
    )

    # Los path de salida se montan como directorios y debemos añadir un nombre al fichero
    credit_train_df.to_csv(os.path.join(args.train_data, "data.csv"), index=False)

    credit_test_df.to_csv(os.path.join(args.test_data, "data.csv"), index=False)

    # paramos el logging
    mlflow.end_run()
    
if __name__ == "__main__":
    main()

Writing ./componentes/data_prep/data_prep.py


In [37]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

data_prep_component = command(
    name="data_prep_credit_defaults",
    display_name="Data preparation para poder entrenar modelos",
    description="Lee un fichero .xls como entrada, hace split a train y test",
    inputs={
        "data": Input(type="uri_folder"),
        "test_train_ratio": Input(type="number"),
    },
    outputs=dict(
        train_data=Output(type="uri_folder", mode="rw_mount"),
        test_data=Output(type="uri_folder", mode="rw_mount"),
    ),
    # Directorio origen donde está el fichero con los pasos a ejecutar
    code=data_prep_src_dir,
    command="""python data_prep.py \
            --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} \
            --train_data ${{outputs.train_data}} --test_data ${{outputs.test_data}} \
            """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
)

In [39]:
import os

train_src_dir = "./components/train"
os.makedirs(train_src_dir, exist_ok=True)

In [40]:
%%writefile {train_src_dir}/train.py
import argparse
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import os
import pandas as pd
import mlflow


def select_first_file(path):
    """Seleccionamos el primer fichero. Asumimos que solo hay uno.
    Args:
        path (str): path al directorio o fichero a elegir
    Returns:
        str: path completo del fichero indicado
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])


# Iniciamos Logging
mlflow.start_run()

# Habilitamos autologging
mlflow.sklearn.autolog()

os.makedirs("./outputs", exist_ok=True)


def main():
    """Función mail del script"""

    # Argumentos para In/Out
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_data", type=str, help="path a los datos para train")
    parser.add_argument("--test_data", type=str, help="path a los datos para test")
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", type=str, help="nombre del modelo")
    parser.add_argument("--model", type=str, help="path al fichero con el modelo")
    args = parser.parse_args()

    #  Los path se montan como directorios y debemos añadir un nombre al fichero
    train_df = pd.read_csv(select_first_file(args.train_data))

    #  Cogemos la columna con la etiqueta / variable target
    y_train = train_df.pop("default payment next month")

    # convertimos el Dataframe a un array 
    X_train = train_df.values

    #  Los path se montan como directorios y debemos añadir un nombre al fichero
    test_df = pd.read_csv(select_first_file(args.test_data))

    # Extracting the label column
    y_test = test_df.pop("default payment next month")

    # convertimos a array
    X_test = test_df.values

    print(f"Entrenamiento con dimensiones: {X_train.shape}")

    clf = GradientBoostingClassifier(
        n_estimators=args.n_estimators, learning_rate=args.learning_rate
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))

    # Registramos el modelo en el workspace de Azure Machine Learning
    print("Registramos el modelo en el workspace usando MLFlow")
    mlflow.sklearn.log_model(
        sk_model=clf,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    # Guardamos el modelo en un fichero
    mlflow.sklearn.save_model(
        sk_model=clf,
        path=os.path.join(args.model, "trained_model"),
    )

    # Finalizamos el Logging
    mlflow.end_run()


if __name__ == "__main__":
    main()

Overwriting ./components/train/train.py


In [41]:
%%writefile {train_src_dir}/train.yml
# <Comienzo definicion componente>
name: train_credit_defaults_model
display_name: Train Credit Defaults Model
# version: 1 # Si no especificamos versión, automáticamente aumentará la versión
type: command
inputs:
  train_data: 
    type: uri_folder
  test_data: 
    type: uri_folder
  learning_rate:
    type: number     
  registered_model_name:
    type: string
outputs:
  model:
    type: uri_folder
code: .
environment:
  # para este paso y como demo del trabajo de TFM, utilizaremos uno de los curated environment, en concreto el que
  # incluye las librerias de sklearn
  azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:21
command: >-
  python train.py 
  --train_data ${{inputs.train_data}} 
  --test_data ${{inputs.test_data}} 
  --learning_rate ${{inputs.learning_rate}}
  --registered_model_name ${{inputs.registered_model_name}} 
  --model ${{outputs.model}}
# </final definicion componente>

Overwriting ./components/train/train.yml


In [43]:
# importamos el Package Component
from azure.ai.ml import load_component

# Cargamos la configuración del componente desde el fichero yaml
train_component = load_component(path=os.path.join(train_src_dir, "train.yml"))

In [44]:
# Registramos el componente en el Workspace
train_component = ml_client.create_or_update(train_component)

print(
    f"Componente {train_component.name} Version: {train_component.version} se ha registrado"
)

Uploading train (0.0 MBs): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 3639/3639 [00:00<00:00, 39292.55it/s]




Componente train_credit_defaults_model Version: 2022-08-15-16-50-29-2935915 se ha registrado


In [45]:
# el decorator DSL le indica a Azure ML que estamos definiendo un pipeline
from azure.ai.ml import dsl, Input, Output


@dsl.pipeline(
    compute=cpu_compute_target,
    description="End to End data_prep-train pipeline",
)
def credit_defaults_pipeline(
    pipeline_job_data_input,
    pipeline_job_test_train_ratio,
    pipeline_job_learning_rate,
    pipeline_job_registered_model_name,
):
    # Utilizamos la funcion data_prep que definimos anteriormente como una llamada en python
    data_prep_job = data_prep_component(
        data=pipeline_job_data_input,
        test_train_ratio=pipeline_job_test_train_ratio,
    )

    # Utilizamos la función de training como una llamada en python
    train_job = train_component(
        train_data=data_prep_job.outputs.train_data,  # Utilizamos las salidas del paso anterior
        test_data=data_prep_job.outputs.test_data,  # Utilizamos las salidas del paso anterior
        learning_rate=pipeline_job_learning_rate,  # Utilizamos los argumentos del pipeline como parametros
        registered_model_name=pipeline_job_registered_model_name,
    )

    # Un pipeline devuelve un diccionario como salida
    return {
        "pipeline_job_train_data": data_prep_job.outputs.train_data,
        "pipeline_job_test_data": data_prep_job.outputs.test_data,
    }


In [46]:
registered_model_name = "credit_defaults_model"

# Ahora instanciamos nuestro pipeline
pipeline = credit_defaults_pipeline(
    # pipeline_job_data_input=credit_data,
    pipeline_job_data_input=Input(type="uri_file", path=web_path),
    pipeline_job_test_train_ratio=0.2,
    pipeline_job_learning_rate=0.25,
    pipeline_job_registered_model_name=registered_model_name,
)

In [47]:
import webbrowser

# submit del job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Nombre que le damos al proyecto
    experiment_name="TFM_validar_metodologia",
)
# Abrimos directamente el pipeline para verlo desde el navegador
webbrowser.open(pipeline_job.services["Studio"].endpoint)

Uploading data_prep (0.0 MBs): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1473/1473 [00:00<00:00, 34080.66it/s]




True

In [55]:
import uuid

# Utilzamos un nombre para el endpoint. 
online_endpoint_name = "credit-endpoint-tfm"

In [56]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# creamos un endpoint Online tipo gestionado
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Endpoint Online para TFM AI",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

endpoint = ml_client.begin_create_or_update(endpoint)

print(f"Endpoint {endpoint.name} state: {endpoint.provisioning_state}")

Endpoint credit-endpoint-tfm state: Succeeded


In [57]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpoint "credit-endpoint-tfm" with provisioning state "Succeeded" is retrieved


In [58]:
# Cogemos la última versión del modelo
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)


In [59]:
# Seleccionamos el modelo con el nombre + versión sacada del paso anterior
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)


# Creamos el despliegue
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

blue_deployment = ml_client.begin_create_or_update(blue_deployment)

Check: endpoint credit-endpoint-tfm exists
Creating/updating online deployment blue 

................................................................................

Done (7m 29s)


In [60]:
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [61]:
%%writefile {deploy_dir}/sample-request.json
{
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "index": [0, 1],
    "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8]
        ]
  }
}

Overwriting ./deploy/sample-request.json


In [80]:
# Lanzamos una request para probar el testing
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)

'[1, 0]'